# Drug Groupings

In [1]:
%run -m ipy_startup
%run -m ipy_logging
%run -m ipy_seaborn
%matplotlib inline
from mgds.data_aggregation import api
from mgds.data_aggregation import database as db
from mgds.data_aggregation import source as src
from mgds.data_aggregation import data_type as dtyp
from mgds.data_aggregation import entity
from mgds.data_modeling import data_broker as dbm
from mgds.data_modeling import training01 as rx_train
from mgds.data_modeling import data as feature_data

In [2]:
df = api.get_raw_genomic_data(src.GDSC_v2, dtyp.DRUG_SENSITIVITY)
df['INDICATOR'] = 1

In [3]:
df.head()

,IC50_RESULTS_ID,CELL_LINE_ID:COSMIC,DRUG_ID:COSMIC,MAX_CONC_MICROMOLAR,LN_IC50,AUC,RMSE,DRUG_NAME,SYNONYMS,TARGET,TARGET_PATHWAY,CELL_LINE_ID,CELL_LINE_ID:MGDS,PRIMARY_SITE:MGDS,PRIMARY_SITE:SOURCE,DRUG_NAME:MGDS,DRUG_NAME:SOURCE,INDICATOR
0,335,924100,1026,1.0,0.717722,0.899410,0.105665,17-AAG,tanespimycin,HSP90,other,22RV1,22RV1,PROSTATE,PROSTATE,17AAG,17-AAG,1
1,336,924100,1028,2.0,2.664100,0.957206,0.178243,VX-702,NaN,p38,JNK and p38 signaling,22RV1,22RV1,PROSTATE,PROSTATE,VX702,VX-702,1
2,337,924100,1029,2.0,3.336828,0.973893,0.079845,AMG-706,AMG-706,"VEGFR, RET, c-KIT, PDGFR",RTK signaling,22RV1,22RV1,PROSTATE,PROSTATE,AMG706,AMG-706,1
3,338,924100,1030,10.0,5.164909,0.977844,0.094228,KU-55933,NaN,ATM,Genome integrity,22RV1,22RV1,PROSTATE,PROSTATE,KU55933,KU-55933,1
4,339,924100,1031,0.2,-4.325309,0.508180,0.090478,Elesclomol,NaN,HSP70,other,22RV1,22RV1,PROSTATE,PROSTATE,ELESCLOMOL,Elesclomol,1


In [5]:
d_path = df[['DRUG_NAME:MGDS', 'TARGET_PATHWAY']].drop_duplicates()

In [6]:
d_path[d_path['DRUG_NAME:MGDS'] == 'NAVITOCLAX']

,DRUG_NAME:MGDS,TARGET_PATHWAY
197,NAVITOCLAX,apoptosis regulation


In [7]:
d_path['TARGET_PATHWAY'].value_counts()

other                             72
RTK signaling                     26
PI3K signaling                    18
ERK MAPK signaling                16
DNA replication                   14
cytoskeleton                      11
cell cycle                        11
mitosis                           10
chromain  histone acetylation     10
ABL signaling                      9
EGFR signaling                     8
Genome integrity                   8
apoptosis regulation               7
TOR signaling                      7
JNK and p38 signaling              6
IGFR signaling                     4
metabolism                         3
chromatin  other                   3
WNT signaling                      3
p53 pathway                        3
chromatin  histone methylation     2
Name: TARGET_PATHWAY, dtype: int64

In [8]:
d_path[d_path['TARGET_PATHWAY'] == 'apoptosis regulation']

,DRUG_NAME:MGDS,TARGET_PATHWAY
32,TW37,apoptosis regulation
53,RTRAIL,apoptosis regulation
89,EMBELIN,apoptosis regulation
91,PAC1,apoptosis regulation
97,OBATOCLAX,apoptosis regulation
144,YM155,apoptosis regulation
197,NAVITOCLAX,apoptosis regulation


In [9]:
"','".join(d_path[d_path['TARGET_PATHWAY'] == 'apoptosis regulation']['DRUG_NAME:MGDS'].values)

"TW37','RTRAIL','EMBELIN','PAC1','OBATOCLAX','YM155','NAVITOCLAX"